In [80]:
import gzip
import json
import numpy as np
import cohere
from cohere.classify import Example
co = cohere.Client('Odj4ClpHI3fobuB9bErJkfzlszJwQ018Z99gUAyE')

In [81]:
def read_gzip():
    text = []
    label = []
    with gzip.open('train.jsonl.gz', 'r') as fin:
        for line in fin:
            data = json.loads(line.decode('utf-8'))
            text.append(data["text"])
            label.append(data["label"])
    return text, label

In [82]:
text, label = read_gzip()
test_text = text[1500:1580]; test_label = label[1500:1580]
train_text = text[0:1000]; train_label = label[0:1000]

In [83]:
def calculate_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [84]:
def classify(train_text, train_label, test_text, test_label):
    examples = []
    for i in range(len(train_text)):
        examples.append(Example(train_text[i], str(train_label[i])))
    inputs = test_text

    response = co.classify(
        model='large',
        inputs=inputs,
        examples=examples)

    accuracy = 0
    for i in range(len(response.classifications)):
        true_label = test_label[i]
        prediction = int(response.classifications[i].prediction)
        confidence = response.classifications[i].confidence
        inputs = response.classifications[i].input
        if true_label != prediction:
            print(f'Input = {inputs}, \n Prediction = {prediction}, Confidence = {confidence}, Label = {true_label} \n')
        else:
            accuracy += 1
    print(accuracy/len(test_label))

In [85]:
classify(train_text, train_label, test_text, test_label)

Input = i hope you can feel the presence of loved ones right by your side cheering you on and wanting the best for you cos youre not on your own you never are d, 
 Prediction = 1, Confidence = 0.9816955, Label = 2 

Input = i already feel it is for the bursts and hesitations of last year to mellow into engaged and rhythmic hops forward like his, 
 Prediction = 4, Confidence = 0.91712064, Label = 1 

Input = i was having a horrible day and decided i would only feel better if i didnt have red hair anymore so i immedietly went to wal mart and found a box of hair color with the description soft dark brown, 
 Prediction = 0, Confidence = 0.94368136, Label = 1 

Input = i really like it a lot and think its a great fit for me and i love talking to the patients and trying to help them feel less nervous or at least that someone cares about them for a few minutes, 
 Prediction = 1, Confidence = 0.99055606, Label = 4 

Input = i was feeling pretty impressed with myself, 
 Prediction = 1, Confiden